<a href="https://colab.research.google.com/github/MillenniumForce/DATA3001/blob/main/deep_learning/Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!pip install polars[pyarrow]
!pip install backports.weakref

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.8 MB 5.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import polars as pl

# Condense features to bytes per packet

In [4]:
polars_df = pl.scan_csv("/content/drive/MyDrive/DATA3001/datascience-cis-2022.csv", parse_dates=True)

In [5]:
cols = polars_df.columns
cols.remove("device_mac")
cols.remove("time")
cols

['FromInternetTCPPort443IPurn:ietf:params:mud:controllerByteCount',
 'FromInternetTCPPort443IPurn:ietf:params:mud:controllerPacketCount',
 'FromInternetTCPPort554IPurn:ietf:params:mud:controllerByteCount',
 'FromInternetTCPPort554IPurn:ietf:params:mud:controllerPacketCount',
 'FromInternetTCPPort80IPurn:ietf:params:mud:controllerByteCount',
 'FromInternetTCPPort80IPurn:ietf:params:mud:controllerPacketCount',
 'FromInternetUDPPort67IPurn:ietf:params:mud:controllerByteCount',
 'FromInternetUDPPort67IPurn:ietf:params:mud:controllerPacketCount',
 'ToInternetRSVPPortAllIPurn:ietf:params:mud:controllerByteCount',
 'ToInternetRSVPPortAllIPurn:ietf:params:mud:controllerPacketCount',
 'ToInternetTCPPort443IPurn:ietf:params:mud:controllerByteCount',
 'ToInternetTCPPort443IPurn:ietf:params:mud:controllerPacketCount',
 'ToInternetTCPPort554IPurn:ietf:params:mud:controllerByteCount',
 'ToInternetTCPPort554IPurn:ietf:params:mud:controllerPacketCount',
 'ToInternetTCPPort80IPurn:ietf:params:mud:contr

In [6]:
to_cols = [col for col in cols if col.startswith("ToInternet")]
from_cols = [col for col in cols if col.startswith("FromInternet") or col.startswith("ToLocal")]

In [14]:
polars_df.with_columns([
    (pl.fold(0, lambda acc, s: acc + s, pl.col(to_cols))).alias("ToInternet"),
    pl.fold(0, lambda acc, s: acc + s, pl.col(from_cols)).alias("FromInternet")
]).fetch(10)#.write_parquet("/content/drive/MyDrive/DATA3001/60min_experiment_to_from_features/experiment_to_from_features.parquet")

#.drop(to_cols + from_cols)

time,FromInternetTCPPort443IPurn:ietf:params:mud:controllerByteCount,FromInternetTCPPort443IPurn:ietf:params:mud:controllerPacketCount,FromInternetTCPPort554IPurn:ietf:params:mud:controllerByteCount,FromInternetTCPPort554IPurn:ietf:params:mud:controllerPacketCount,FromInternetTCPPort80IPurn:ietf:params:mud:controllerByteCount,FromInternetTCPPort80IPurn:ietf:params:mud:controllerPacketCount,FromInternetUDPPort67IPurn:ietf:params:mud:controllerByteCount,FromInternetUDPPort67IPurn:ietf:params:mud:controllerPacketCount,ToInternetRSVPPortAllIPurn:ietf:params:mud:controllerByteCount,ToInternetRSVPPortAllIPurn:ietf:params:mud:controllerPacketCount,ToInternetTCPPort443IPurn:ietf:params:mud:controllerByteCount,ToInternetTCPPort443IPurn:ietf:params:mud:controllerPacketCount,ToInternetTCPPort554IPurn:ietf:params:mud:controllerByteCount,ToInternetTCPPort554IPurn:ietf:params:mud:controllerPacketCount,ToInternetTCPPort80IPurn:ietf:params:mud:controllerByteCount,ToInternetTCPPort80IPurn:ietf:params:mud:controllerPacketCount,ToInternetUDPPort1024IPurn:ietf:params:mud:controllerByteCount,ToInternetUDPPort1024IPurn:ietf:params:mud:controllerPacketCount,ToLocalUDPPort5353IP224.0.0.251/32ByteCount,ToLocalUDPPort5353IP224.0.0.251/32PacketCount,ToLocalUDPPort67IP255.255.255.255/32ByteCount,ToLocalUDPPort67IP255.255.255.255/32PacketCount,device_mac,ToInternet,FromInternet
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64
1662078299417000000,0,0,0,0,0,0,0,0,3568,8,0,0,0,0,395738,957,12625992,9093,0,0,0,0,"""bd:0d:8e:c5""",13035356,0
1662078299417000000,0,0,0,0,71161,266,0,0,1784,4,0,0,14674098,11246,68385,150,0,0,0,0,0,0,"""bd:24:bb:1e""",14755667,71427
1662078299417000000,0,0,0,0,25880,91,0,0,3568,8,0,0,0,0,103786,243,0,0,0,0,0,0,"""bd:0d:af:0b""",107605,25971
1662078299417000000,0,0,0,0,0,0,0,0,3568,8,0,0,0,0,29638,81,0,0,0,0,0,0,"""ee:dd:34:1d""",33295,0
1662078299417000000,0,0,0,0,0,0,1400,4,1784,4,0,0,13021325,9857,0,0,0,0,0,0,906,2,"""ee:dd:39:08""",13032970,2312
1662078299417000000,0,0,0,0,76946,287,0,0,1784,4,0,0,13024382,9717,68455,149,0,0,0,0,0,0,"""ee:dd:39:09""",13104491,77233
1662078299417000000,0,0,0,0,0,0,0,0,3568,8,0,0,0,0,32520,87,0,0,0,0,0,0,"""bd:24:bb:45""",36183,0
1662078299417000000,0,0,0,0,78665,291,0,0,1784,4,0,0,0,0,32514,87,0,0,0,0,0,0,"""bd:24:bb:58""",34389,78956
1662078299417000000,0,0,0,0,75162,281,0,0,1784,4,0,0,0,0,103805,243,0,0,0,0,0,0,"""bd:12:72:47""",105836,75443


In [ ]:
# Using Polars otherwise Pandas crashes

for byte, packet in zip(cols[::2], cols[1::2]):
  new_col_name = byte.replace("ByteCount", "BytesPerPacket")
  polars_df = polars_df.with_columns([
    (pl.col(byte) / pl.col(packet)).alias(new_col_name)
  ])
  polars_df = polars_df.drop([byte, packet])

polars_df = polars_df.fill_null(0)
# polars_df.collect().write_parquet("/content/drive/MyDrive/DATA3001/bytes_per_packet_features.parquet")

In [ ]:
processed_cols = polars_df.columns

pairs = [
    ("FromInternetTCPPort443IPurn:ietf:params:mud:controllerBytesPerPacket", "ToInternetTCPPort443IPurn:ietf:params:mud:controllerBytesPerPacket"),
    ("FromInternetTCPPort554IPurn:ietf:params:mud:controllerBytesPerPacket", "ToInternetTCPPort554IPurn:ietf:params:mud:controllerBytesPerPacket"),
    ("FromInternetTCPPort80IPurn:ietf:params:mud:controllerBytesPerPacket", "ToInternetTCPPort80IPurn:ietf:params:mud:controllerBytesPerPacket"),
    ("FromInternetUDPPort67IPurn:ietf:params:mud:controllerBytesPerPacket",  "ToLocalUDPPort67IP255.255.255.255/32BytesPerPacket")
]

for col1, col2 in pairs:
  new_col_name = col1.replace("FromInternet", "ToFromDiff")
  polars_df = polars_df.with_columns([
    (pl.col(col1) - pl.col(col2)).alias(new_col_name)
  ])
  polars_df = polars_df.drop([col1, col2])

polars_df.collect().write_parquet("/content/drive/MyDrive/DATA3001/diff_bytes_per_packet_features.parquet")

KeyboardInterrupt: ignored

# Load Features

In [7]:
df = pd.read_parquet("/content/drive/MyDrive/DATA3001/diff_bytes_per_packet_features.parquet")
df.time = pd.to_datetime(df.time, unit="ns")
df = df.set_index("time")

In [8]:
print(df.shape)
df.head(5)

(14376698, 8)


,device_mac,ToInternetRSVPPortAllIPurn:ietf:params:mud:controllerBytesPerPacket,ToInternetUDPPort1024IPurn:ietf:params:mud:controllerBytesPerPacket,ToLocalUDPPort5353IP224.0.0.251/32BytesPerPacket,ToFromDiffTCPPort443IPurn:ietf:params:mud:controllerBytesPerPacket,ToFromDiffTCPPort554IPurn:ietf:params:mud:controllerBytesPerPacket,ToFromDiffTCPPort80IPurn:ietf:params:mud:controllerBytesPerPacket,ToFromDiffUDPPort67IPurn:ietf:params:mud:controllerBytesPerPacket
time,,,,,,,,
2022-09-02 00:24:59.417,bd:0d:8e:c5,446,1388,0,0,0,-413,0
2022-09-02 00:24:59.417,bd:24:bb:1e,446,0,0,0,-1304,-188,0
2022-09-02 00:24:59.417,bd:0d:af:0b,446,0,0,0,0,-143,0
2022-09-02 00:24:59.417,ee:dd:34:1d,446,0,0,0,0,-365,0
2022-09-02 00:24:59.417,ee:dd:39:08,446,0,0,0,-1321,0,-103


# Feature generation

features = time steps over $k$ window size (resample)

outcome = Y/N if the data belongs to the chosen network flow

In [ ]:
chosen_flow = "FromInternetTCPPort80IPurn:ietf:params:mud:controllerByteCount"
window_size = 60 #i.e. 1 hour (use pca later?)

sample = df[["device_mac", "FromInternetTCPPort80IPurn:ietf:params:mud:controllerByteCount"]]
sample

,device_mac,FromInternetTCPPort80IPurn:ietf:params:mud:controllerByteCount
time,,
2022-09-02 00:24:59.417,bd:0d:8e:c5,0
2022-09-02 00:24:59.417,bd:24:bb:1e,71161
2022-09-02 00:24:59.417,bd:0d:af:0b,25880
2022-09-02 00:24:59.417,ee:dd:34:1d,0
2022-09-02 00:24:59.417,ee:dd:39:08,0
...,...,...
2022-09-16 00:23:55.620,bd:1c:cf:14,0
2022-09-16 00:23:55.620,21:fa:29:50,1934
2022-09-16 00:23:55.620,ee:dd:49:33,74945


In [ ]:
sample = sample.groupby("device_mac").resample("1min").sum().unstack()

In [ ]:
sample

In [ ]:
sample.columns = sample.columns.droplevel()

In [ ]:
sample.columns = np.arange(0, len(sample.columns))

In [ ]:
sample

,0,1,2,3,4,5,6,7,8,9,...,20150,20151,20152,20153,20154,20155,20156,20157,20158,20159
device_mac,,,,,,,,,,,,,,,,,,,,,
21:fa:27:56,967.0,967.0,967.0,967.0,0.0,967.0,967.0,967.0,967.0,967.0,...,967.0,967.0,967.0,967.0,967.0,2468.0,967.0,967.0,967.0,NaN
21:fa:27:82,71991.0,71310.0,78759.0,70941.0,72726.0,71035.0,75039.0,76382.0,72848.0,75039.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21:fa:29:50,967.0,2887.0,967.0,967.0,967.0,967.0,967.0,967.0,967.0,967.0,...,1934.0,1934.0,1934.0,7386.0,3854.0,3854.0,3854.0,1934.0,1934.0,1934.0
21:fa:2d:cf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21:fa:2e:47,45199.0,49109.0,49391.0,45387.0,50631.0,46433.0,50605.0,45387.0,49203.0,49297.0,...,72754.0,74945.0,70663.0,69121.0,74945.0,70941.0,72726.0,70941.0,74945.0,70941.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ee:dd:4e:5f,46556.0,48867.0,46838.0,48676.0,44787.0,53547.0,47375.0,46363.0,46826.0,43661.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ee:dd:4e:60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ee:dd:4f:4c,76456.0,70402.0,73066.0,74976.0,76817.0,72249.0,76955.0,71161.0,76788.0,78879.0,...,25905.0,25905.0,25905.0,25905.0,25905.0,25905.0,25905.0,0.0,25905.0,25905.0


In [ ]:
sample.iloc[:, 60:60 + window_size]

In [ ]:
batches = []

for i in range(0, len(sample.columns) - window_size, window_size):
  batches.append(sample.iloc[:, i:i+window_size].to_numpy())
  # print(i, i + window_size)

batches = np.concatenate(batches)

In [ ]:
batches = pd.DataFrame(batches)
batches["flow_type"] = chosen_flow

In [ ]:
batches

In [ ]:
# Total number of rows to expect
252255 * 22

5549610

# Split data train, val, test (0.6, 0.2, 0.2)

In [9]:
n = len(df)
train_df = df[0:int(n*0.6)]
val_df = df[int(n*0.6):int(n*0.8)]
test_df = df[int(n*0.8):]

In [10]:
train_df.min(axis=0)

device_mac                                                             21:fa:27:56
ToInternetRSVPPortAllIPurn:ietf:params:mud:controllerBytesPerPacket              0
ToInternetUDPPort1024IPurn:ietf:params:mud:controllerBytesPerPacket              0
ToLocalUDPPort5353IP224.0.0.251/32BytesPerPacket                                 0
ToFromDiffTCPPort443IPurn:ietf:params:mud:controllerBytesPerPacket               0
ToFromDiffTCPPort554IPurn:ietf:params:mud:controllerBytesPerPacket           -1496
ToFromDiffTCPPort80IPurn:ietf:params:mud:controllerBytesPerPacket            -1522
ToFromDiffUDPPort67IPurn:ietf:params:mud:controllerBytesPerPacket             -598
dtype: object

In [11]:
del df

In [12]:
cols = train_df.columns.to_list()
cols.remove("device_mac")
cols

['ToInternetRSVPPortAllIPurn:ietf:params:mud:controllerBytesPerPacket',
 'ToInternetUDPPort1024IPurn:ietf:params:mud:controllerBytesPerPacket',
 'ToLocalUDPPort5353IP224.0.0.251/32BytesPerPacket',
 'ToFromDiffTCPPort443IPurn:ietf:params:mud:controllerBytesPerPacket',
 'ToFromDiffTCPPort554IPurn:ietf:params:mud:controllerBytesPerPacket',
 'ToFromDiffTCPPort80IPurn:ietf:params:mud:controllerBytesPerPacket',
 'ToFromDiffUDPPort67IPurn:ietf:params:mud:controllerBytesPerPacket']

In [13]:
train_df

,device_mac,ToInternetRSVPPortAllIPurn:ietf:params:mud:controllerBytesPerPacket,ToInternetUDPPort1024IPurn:ietf:params:mud:controllerBytesPerPacket,ToLocalUDPPort5353IP224.0.0.251/32BytesPerPacket,ToFromDiffTCPPort443IPurn:ietf:params:mud:controllerBytesPerPacket,ToFromDiffTCPPort554IPurn:ietf:params:mud:controllerBytesPerPacket,ToFromDiffTCPPort80IPurn:ietf:params:mud:controllerBytesPerPacket,ToFromDiffUDPPort67IPurn:ietf:params:mud:controllerBytesPerPacket
time,,,,,,,,
2022-09-02 00:24:59.417,bd:0d:8e:c5,446,1388,0,0,0,-413,0
2022-09-02 00:24:59.417,bd:24:bb:1e,446,0,0,0,-1304,-188,0
2022-09-02 00:24:59.417,bd:0d:af:0b,446,0,0,0,0,-143,0
2022-09-02 00:24:59.417,ee:dd:34:1d,446,0,0,0,0,-365,0
2022-09-02 00:24:59.417,ee:dd:39:08,446,0,0,0,-1321,0,-103
...,...,...,...,...,...,...,...,...
2022-09-10 09:55:01.745,bd:1f:b1:92,0,0,0,0,0,74,0
2022-09-10 09:55:01.745,bd:1c:cb:3a,446,1386,0,0,0,0,0
2022-09-10 09:55:01.745,bd:24:bb:33,446,0,0,0,-1291,267,0


# Bulk compute all features

In [14]:
DEVICE_MAC = "device_mac"


def construct_flow_dataframe(df, flow_type, window_size=5):
  flow_df = df[[DEVICE_MAC, flow_type]]

  # Resample to 1 minute to ensure that any time discrepencies are accounted for
  flow_df.index = flow_df.index.round("min")

  # Pivot to set time as columns
  flow_df = flow_df.reset_index() \
                   .pivot(index=DEVICE_MAC, values=flow_type, columns="time")

  # Linear interploation to fill nans
  flow_df = flow_df.interpolate(axis="columns", limit_direction="both")

  # Replace timestamps with ints for easy indexing of windows
  flow_df.columns = np.arange(0, len(flow_df.columns))

  # Using a rolling window type scheme to extract data
  # For a 60min rolling window each batch with contain 60 minutes each
  # independent from each other (this is to avoid huge amounts of data
  # from a rolling window of 0-59, 1-60, 2-61 etc. use instead 0-59, 60-119...)
  batches = []
  for i in range(0, len(flow_df.columns) - window_size, window_size):
    batches.append(flow_df.iloc[:, i:i+window_size].to_numpy())
    # print(i, i + window_size)
  batches = np.concatenate(batches)

  # Construct final dataframe
  flow_df = pd.DataFrame(batches)
  flow_df["flow_type"] = flow_type

  # ensure cleanup for memory efficiency
  del batches

  return flow_df


In [12]:
construct_flow_dataframe(train_df, "ToInternetRSVPPortAllIPurn:ietf:params:mud:controllerBytesPerPacket")

,0,1,2,3,4,flow_type
0,446.0,446.0,446.0,446.0,446.0,ToInternetRSVPPortAllIPurn:ietf:params:mud:con...
1,446.0,446.0,446.0,446.0,446.0,ToInternetRSVPPortAllIPurn:ietf:params:mud:con...
2,450.0,378.0,378.0,450.0,450.0,ToInternetRSVPPortAllIPurn:ietf:params:mud:con...
3,0.0,0.0,0.0,0.0,0.0,ToInternetRSVPPortAllIPurn:ietf:params:mud:con...
4,446.0,446.0,446.0,446.0,446.0,ToInternetRSVPPortAllIPurn:ietf:params:mud:con...
...,...,...,...,...,...,...
1750720,446.0,446.0,446.0,446.0,446.0,ToInternetRSVPPortAllIPurn:ietf:params:mud:con...
1750721,446.0,446.0,446.0,446.0,446.0,ToInternetRSVPPortAllIPurn:ietf:params:mud:con...
1750722,446.0,446.0,446.0,446.0,446.0,ToInternetRSVPPortAllIPurn:ietf:params:mud:con...
1750723,290.0,293.0,286.0,300.0,293.0,ToInternetRSVPPortAllIPurn:ietf:params:mud:con...


In [14]:
from multiprocessing import Pool
from tqdm.notebook import tqdm

In [15]:
from multiprocessing import cpu_count

In [15]:
cpu_count()

2

In [38]:
flow_datasets = [construct_flow_dataframe(val_df, col) for col in tqdm(cols)]

  0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
from functools import partial
from tqdm.contrib.concurrent import process_map

flow_datasets = process_map(partial(construct_flow_dataframe, train_df), 
                            cols,
                            max_workers=1, 
                            chunksize=5)

In [40]:
flow_datasets[0]

,0,1,2,3,4,flow_type
0,-0.231034,-0.231034,-0.231034,-0.231034,-0.231034,ToInternetRSVPPortAllIPurn:ietf:params:mud:con...
1,-0.231034,-0.231034,-0.231034,-0.231034,-0.231034,ToInternetRSVPPortAllIPurn:ietf:params:mud:con...
2,-0.348276,-0.348276,-0.224138,-0.431034,-0.327586,ToInternetRSVPPortAllIPurn:ietf:params:mud:con...
3,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,ToInternetRSVPPortAllIPurn:ietf:params:mud:con...
4,-0.231034,-0.231034,-0.231034,-0.231034,-0.231034,ToInternetRSVPPortAllIPurn:ietf:params:mud:con...
...,...,...,...,...,...,...
583570,-0.231034,-0.231034,-0.231034,-0.231034,-0.231034,ToInternetRSVPPortAllIPurn:ietf:params:mud:con...
583571,-0.231034,-0.231034,-0.334483,-0.231034,-0.231034,ToInternetRSVPPortAllIPurn:ietf:params:mud:con...
583572,-0.231034,-0.231034,-0.231034,-0.231034,-0.231034,ToInternetRSVPPortAllIPurn:ietf:params:mud:con...
583573,-0.515517,-0.515517,-0.518966,-0.515517,-0.515517,ToInternetRSVPPortAllIPurn:ietf:params:mud:con...


In [39]:
fld = pd.concat(flow_datasets, axis=0)
fld.columns = fld.columns.astype(str)
fld.to_parquet("/content/drive/MyDrive/DATA3001/5_min_bytes_per_packet_feats_diff/val.parquet")

In [ ]:
fld.isna().sum().any()

False

In [41]:
fld

,0,1,2,3,4,flow_type
0,-0.231034,-0.231034,-0.231034,-0.231034,-0.231034,ToInternetRSVPPortAllIPurn:ietf:params:mud:con...
1,-0.231034,-0.231034,-0.231034,-0.231034,-0.231034,ToInternetRSVPPortAllIPurn:ietf:params:mud:con...
2,-0.348276,-0.348276,-0.224138,-0.431034,-0.327586,ToInternetRSVPPortAllIPurn:ietf:params:mud:con...
3,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,ToInternetRSVPPortAllIPurn:ietf:params:mud:con...
4,-0.231034,-0.231034,-0.231034,-0.231034,-0.231034,ToInternetRSVPPortAllIPurn:ietf:params:mud:con...
...,...,...,...,...,...,...
583570,0.261603,0.261603,0.261603,0.261603,0.044304,ToFromDiffUDPPort67IPurn:ietf:params:mud:contr...
583571,0.044304,0.261603,0.261603,0.261603,0.261603,ToFromDiffUDPPort67IPurn:ietf:params:mud:contr...
583572,0.044304,0.261603,0.261603,0.261603,0.261603,ToFromDiffUDPPort67IPurn:ietf:params:mud:contr...
583573,0.261603,0.261603,0.261603,0.261603,0.261603,ToFromDiffUDPPort67IPurn:ietf:params:mud:contr...


# Bulk compute

In [15]:
import os

def save(flow_datasets, mins, train_test_or_val):
    fld = pd.concat(flow_datasets, axis=0)
    fld.columns = fld.columns.astype(str)
    folder = f"/content/drive/MyDrive/DATA3001/{mins}_min_bytes_per_packet_feats_diff"
    if not os.path.isdir(folder):
      os.makedirs(folder)
    fld.to_parquet(f"{folder}/{train_test_or_val}.parquet")

# for mins in tqdm((5, 15, 30, 60)):
for mins in tqdm((1, 2, 3, 4)):
  flow_datasets = [construct_flow_dataframe(train_df, col, window_size=mins) for col in tqdm(cols)]
  save(flow_datasets, mins, "train")

  flow_datasets = [construct_flow_dataframe(test_df, col, window_size=mins) for col in tqdm(cols)]
  save(flow_datasets, mins, "test")

  flow_datasets = [construct_flow_dataframe(val_df, col, window_size=mins) for col in tqdm(cols)]
  save(flow_datasets, mins, "val")

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

...

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

# Inspect data

In [23]:
df = pd.read_parquet("/content/drive/MyDrive/DATA3001/5_min_bytes_per_packet_feats_diff/test.parquet")

In [24]:
df

,0,1,2,3,4,flow_type
0,446.0,446.0,446.0,446.0,446.0,ToInternetRSVPPortAllIPurn:ietf:params:mud:con...
1,446.0,446.0,446.0,446.0,446.0,ToInternetRSVPPortAllIPurn:ietf:params:mud:con...
2,450.0,378.0,450.0,390.0,450.0,ToInternetRSVPPortAllIPurn:ietf:params:mud:con...
3,0.0,0.0,0.0,0.0,0.0,ToInternetRSVPPortAllIPurn:ietf:params:mud:con...
4,446.0,446.0,446.0,446.0,446.0,ToInternetRSVPPortAllIPurn:ietf:params:mud:con...
...,...,...,...,...,...,...
583570,0.0,0.0,0.0,0.0,0.0,ToFromDiffUDPPort67IPurn:ietf:params:mud:contr...
583571,0.0,-103.0,-103.0,-103.0,0.0,ToFromDiffUDPPort67IPurn:ietf:params:mud:contr...
583572,0.0,-103.0,-103.0,-103.0,0.0,ToFromDiffUDPPort67IPurn:ietf:params:mud:contr...
583573,-103.0,-103.0,-103.0,0.0,0.0,ToFromDiffUDPPort67IPurn:ietf:params:mud:contr...


In [12]:
df.shape[0] - df[df != 0].dropna().shape[0]

2787957

In [7]:
from sklearn.decomposition import PCA

In [15]:
pca = PCA()
pd.DataFrame(pca.fit_transform(df.drop(columns="flow_type")))

,0,1,2,3,4
0,-793.911080,0.044067,0.205705,0.054885,-0.019396
1,-793.911080,0.044067,0.205705,0.054885,-0.019396
2,-743.814311,4.191051,45.033200,-56.683827,-7.253381
3,203.375195,-0.009144,-0.072239,-0.014766,-0.006713
4,-793.911080,0.044067,0.205705,0.054885,-0.019396
...,...,...,...,...,...
4085020,203.375195,-0.009144,-0.072239,-0.014766,-0.006713
4085021,341.595323,4.417411,112.435665,-6.670273,0.520116
4085022,341.595323,4.417411,112.435665,-6.670273,0.520116
4085023,341.577280,103.260449,16.938170,36.704259,-20.620644
